In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from glasses.nn.models.classification.fishnet import *
from glasses.nn.models.classification.resnet import *
from glasses.nn.models.classification.densenet import *

from torchsummary import summary
import torch
import torch.nn as nn

In [14]:
summary(DenseNet.densenet161().cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 96, 112, 112]          14,112
       BatchNorm2d-2         [-1, 96, 112, 112]             192
              ReLU-3         [-1, 96, 112, 112]               0
         MaxPool2d-4           [-1, 96, 56, 56]               0
       BatchNorm2d-5           [-1, 96, 56, 56]             192
              ReLU-6           [-1, 96, 56, 56]               0
         Conv2dPad-7          [-1, 192, 56, 56]          18,432
       BatchNorm2d-8          [-1, 192, 56, 56]             384
              ReLU-9          [-1, 192, 56, 56]               0
        Conv2dPad-10           [-1, 48, 56, 56]          82,944
DenseBottleNeckBlock-11          [-1, 144, 56, 56]               0
      BatchNorm2d-12          [-1, 144, 56, 56]             288
             ReLU-13          [-1, 144, 56, 56]               0
        Conv2dPad-14          [-1, 1

(tensor(28681000), tensor(28681000), tensor(109.4093), tensor(811.7996))

In [ ]:
ResNetBottleneckPreActBlock(64, 256, stride=2)

In [ ]:
FishNetBottleNeck(32, 64)

In [ ]:
ResNetBottleneckPreActBlock(32, 256, shortcut=FishNetShortCut)

In [ ]:
model = ResNet.resnet50()
model

In [ ]:
model = ResNet.resnet50()

summary(model.cuda(), (3,224,224))

In [ ]:
from torchvision.models import resnet50

model = resnet50(False)

summary(model.cuda(), (3,224,224))

In [ ]:
list(map(lambda x: x* 4, [64, 128, 256, 512]))

In [ ]:
FishNet.fishnet99()

In [ ]:
from glasses.nn.models.classification import SpatialSE

block = lambda in_ch, out_ch: nn.Sequential(FishNetBottleNeck(in_ch, out_ch), SpatialSE(out_ch))

block(32, 64)

In [ ]:
FishNet.fishnet99(activation = nn.SELU)
# change number of classes (default is 1000 )
FishNet.fishnet99(n_classes=100)
# pass a different block
block = lambda in_ch, out_ch, **kwargs: nn.Sequential(FishNetBottleNeck(in_ch, out_ch), SpatialSE(out_ch))
FishNet.fishnet99(block=block)